In [15]:
import requests
import pandas as pd
import defs
from datetime import datetime
import pytz

/Users/egrados/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Debo obtener la data de ESTA semana. Parametros en RFC3339.  
Considerando que solo obtengo 500 velas por llamada de la API y que cada dia tengo 1440 velasde M1, tengo que hacer 3 llamadas de la API por cada dia de la semana.
- Inicio: 2024-06-24T04:00:00.000000000Z  #hora de NY UTC-4 para esta semana
- Inicio: 2024-06-24T04:00:00.000000000Z  
  
El API nos regresa de forma nativa la hora UTC-0 con la Z, por lo que la convertiremos a UTC-4 (hora actual de NY).  
Notar que en RFC339, la variable Z se usa para indicar que es UTC+0, por lo que la usare siempre (https://medium.easyread.co/understanding-about-rfc-3339-for-datetime-formatting-in-software-engineering-940aa5d5f68a).  


In [30]:
#Funcion para obtener hora actual de NY en formato Z UTC-0
import pytz
from datetime import datetime

def obtener_hora_nueva_york_rfc3339():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Formatear la fecha y hora en el formato deseado
    hora_formateada = ahora_ny.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_formateada

# Ejemplo de uso
hora_actual_ny = obtener_hora_nueva_york_rfc3339()
print("Hora actual en Nueva York:", hora_actual_ny)


Hora actual en Nueva York: 2024-06-26T15:03:33.486168Z


In [20]:
from datetime import datetime, timedelta

def restar_siete_dias(fecha_hora_rfc3339):
    # Convertir la cadena RFC3339 a un objeto datetime
    fecha_hora = datetime.strptime(fecha_hora_rfc3339, "%Y-%m-%dT%H:%M:%S.%fZ")
    
    # Restar 7 días a la fecha y hora
    fecha_hora_menos_siete_dias = fecha_hora - timedelta(days=7)
    
    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    nueva_fecha_hora_rfc3339 = fecha_hora_menos_siete_dias.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    return nueva_fecha_hora_rfc3339

# Ejemplo de uso
fecha_hora_original = "2023-05-25T14:32:15.123456Z"
fecha_hora_menos_siete_dias = restar_siete_dias(fecha_hora_original)
print("Fecha y hora original:", fecha_hora_original)
print("Fecha y hora 7 días antes:", fecha_hora_menos_siete_dias)


Fecha y hora original: 2023-05-25T14:32:15.123456Z
Fecha y hora 7 días antes: 2023-05-18T14:32:15.123456Z


Voy a pedile a a API que me entregue las velas los ultimos 7 dias siguiendo la hora de ny que es -4 . Yo manualmente le pongo la hora de inicio para simplificar este paso, luego tendra que ser automatizado.

In [19]:
now_time = obtener_hora_nueva_york_rfc3339()


In [ ]:
def testing(x):
    return x*x
